In [48]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [1]:
import time

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image

In [228]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()  # Important, otherwise will throw an error

    self.c1 = torch.nn.Conv2d(3, 32, 5, padding=2)
    self.p1 = torch.nn.MaxPool2d(2)
    
    self.c2 = torch.nn.Conv2d(32, 64, 5, padding=2)
    self.p2 = torch.nn.MaxPool2d(2)

    self.c3 = torch.nn.Conv2d(64, 64, 5, padding=2)
    self.p3 = torch.nn.MaxPool2d(2)
    
    self.l1 = torch.nn.Linear(4 * 4 * 64, 1000)
    self.l2 = torch.nn.Linear(1000, 10)

  def forward(self, x):
    
    c1_i = x
    c1_o = self.c1(c1_i)
    x = F.relu(c1_o)
    x = self.p1(x)
    
    #c1_i.retain_grad()
    #c1_o.retain_grad()
    
    c2_i = x
    c2_o = self.c2(c2_i)
    x = F.relu(c2_o)
    x = self.p2(x)
    
    c3_i = x
    c3_o = self.c3(c3_i)
    x = F.relu(c3_o)
    x = self.p3(x)
    
    x = torch.flatten(x, start_dim=1)
    
    x = F.relu(self.l1(x))
    
    x = self.l2(x)
    
    return c1_i, c1_o, c2_i, c2_o, c3_i, c3_o, x


net = CNN()
print(net)

CNN(
  (c1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (p2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (p3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l1): Linear(in_features=1024, out_features=1000, bias=True)
  (l2): Linear(in_features=1000, out_features=10, bias=True)
)


In [229]:
fake_input = torch.randn(2, 3, 32, 32)


c1_i, c1_o, c2_i, c2_o, c3_i, c3_o, outputs = net(fake_input)
net.zero_grad()

outputs

tensor([[ 0.0234, -0.0230,  0.0264, -0.0998, -0.0421,  0.0030, -0.0220,  0.0259,
         -0.0675, -0.0316],
        [ 0.0126, -0.0407,  0.0282, -0.0782, -0.0266,  0.0091, -0.0212,  0.0132,
         -0.0635, -0.0215]], grad_fn=<AddmmBackward0>)

In [230]:
def Grad(inputs, outputs) :
    outputs_sum = outputs.sum()
    
    inputs.retain_grad()
    outputs.retain_grad()
    outputs_sum.backward(retain_graph=True)
    return torch.abs(inputs.grad)

In [ ]:
Grad(c2_i, c2_o)

In [24]:
def GradxInput(inputs, outputs) :
    Grad_val = Grad(inputs, outputs)
    return Grad_val * inputs

In [ ]:
GradxInput(c2_i, c2_o)

In [231]:
fake_prior = torch.randn(2, 64, 64, requires_grad=True)


In [232]:
def pal_loss(inputs, outputs, prior, attribution_method, channel_strategy=None) :
    attribution_map = attribution_method(inputs, outputs)
    
    if channel_strategy == "half_mean" :
        nb_class = attribution_map.shape[1]
        attribution_map[:, nb_class/2:, :, :]
    
    if channel_strategy == "half_mean" or channel_strategy == "mean" :
        attribution_map = attribution_map.mean(1).unsqueeze(1)
    
    attribution_map_resize = transforms.Resize(attribution_map.shape[-2:])
    
    prior = attribution_map_resize(prior)
    
    std = attribution_map.view(attribution_map.size(0), -1).std(1)
    mean = attribution_map.view(attribution_map.size(0), -1).mean(1)
    
    
    res = (attribution_map - mean.view(-1, 1, 1, 1)) / std.view(-1, 1, 1, 1)
    res = res * prior.unsqueeze(1)
    
    res = res.view(res.size(0), -1).sum(1)
    res = -res
    return res.mean()
    

In [233]:
loss = pal_loss(c2_i, c2_o, fake_prior, GradxInput)
net.zero_grad()
loss.backward()